In [1]:
import pandas as pd

In [2]:
movies_df = pd.read_csv("Data/movies.csv")

ratings_df = pd.read_csv("Data/ratings.csv")

In [3]:
movies_df

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
86532,288967,State of Siege: Temple Attack (2021),Action|Drama
86533,288971,Ouija Japan (2021),Action|Horror
86534,288975,The Men Who Made the Movies: Howard Hawks (1973),Documentary
86535,288977,Skinford: Death Sentence (2023),Crime|Thriller


In [4]:
ratings_df

,userId,movieId,rating,timestamp
0,1,1,4.0,1225734739
1,1,110,4.0,1225865086
2,1,158,4.0,1225733503
3,1,260,4.5,1225735204
4,1,356,5.0,1225735119
...,...,...,...,...
33832157,330975,8340,2.0,1091583256
33832158,330975,8493,2.5,1091585709
33832159,330975,8622,4.0,1091581777
33832160,330975,8665,3.0,1091581765


In [5]:
ratings_per_movie = ratings_df.groupby('movieId')['rating'].count().reset_index()
ratings_per_movie.columns = ['movieId', 'ratings_count']
ratings_per_movie.head()

,movieId,ratings_count
0,1,76813
1,2,30209
2,3,15820
3,4,3028
4,5,15801


In [6]:
ratings_df = pd.merge(ratings_df, ratings_per_movie, on='movieId', how= 'inner')
ratings_df

,userId,movieId,rating,timestamp,ratings_count
0,1,1,4.0,1225734739,76813
1,2,1,5.0,835815971,76813
2,7,1,4.0,974518024,76813
3,10,1,3.0,1430666394,76813
4,12,1,5.0,862500738,76813
...,...,...,...,...,...
33832157,330842,196133,0.5,1671069205,1
33832158,330852,217236,2.0,1594246738,1
33832159,330904,228179,4.5,1667167519,1
33832160,330904,261553,3.5,1667165898,1


In [10]:
ratings_df = ratings_df[ratings_df['ratings_count']>= 50000]
ratings_df

,userId,movieId,rating,timestamp,ratings_count
0,1,1,4.0,1225734739,76813
1,2,1,5.0,835815971,76813
2,7,1,4.0,974518024,76813
3,10,1,3.0,1430666394,76813
4,12,1,5.0,862500738,76813
...,...,...,...,...,...
16055216,330949,608,4.0,1621955525,61977
16055217,330952,608,5.0,862690931,61977
16055218,330954,608,4.0,867041945,61977
16055219,330964,608,4.0,1649130836,61977


In [14]:
drop_columns = ratings_df.drop(columns=['timestamp','ratings_count'])
clean_ratings_over_50000 = drop_columns.sort_values(by='userId')

In [16]:
clean_ratings_over_50000.to_csv('clean_ratings.csv')

In [17]:
new_movies_df = movies_df[movies_df['movieId'].isin(clean_ratings_over_50000['movieId'])]
new_movies_df

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
31,32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
46,47,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
49,50,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
108,110,Braveheart (1995),Action|Drama|War
148,150,Apollo 13 (1995),Adventure|Drama|IMAX
257,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi
292,296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
314,318,"Shawshank Redemption, The (1994)",Crime|Drama
351,356,Forrest Gump (1994),Comedy|Drama|Romance|War
